In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **PROJETO APLICADO**

**Nomes dos Integrantes:** 

- Fernanda de Siqueira Teixeira
- Henrique Mendes

# **Entendimento do negócio**

**TEMA ESCOLHIDO**: Ciência de Dados Comportamentais

<i>"A combinação multidisciplinar de Ciência de Dados e análise de comportamento humano que possibilita tomar decisões impactantes"</i>

**Base escolhida**: https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store

### **Problema**

Este arquivo contém dados de comportamento por 2 meses (outubro de 2019 e novembro de 2019) de uma grande loja online multicategoria.

Cada linha do arquivo representa um evento. Todos os eventos são relacionados a produtos e usuários. Cada evento é como uma relação muitos para muitos entre produtos e usuários.

Devido ao tamanho da base, vamos analisar nesse trabalho apenas o mês de Outubro.

### **Justificativa**

- conjunto de dados validado;
- área escolhida;

## **Objetivo**:

- Prever no momento da adição ao carrinho de compras se o usuário comprará um determinado produto ou não

# **Entendimento dos dados**

### **Descrição dos Dados**

- **event_time**: Horário em que o evento aconteceu (em UTC).
- **event_type**: Representa o tipo de evento.
- **product_id**: ID de um produto
- **category_id**: ID da categoria do produto
- **category_code**: Taxonomia da categoria do produto (codinome), se possível. Geralmente presente para categorias significativas e ignorado para diferentes tipos de acessórios.
- **brand**: Cadeia de caracteres com o nome da marca. 
- **price**: Preço flutuante de um produto. Presente.
- **user_id**: ID de usuário permanente.
- **user_session**: ID de sessão do usuário temporário. O mesmo para cada sessão de usuário. É alterado toda vez que o usuário volta à loja online após uma longa pausa.


Os eventos podem ser:

- view: um usuário visualizou um produto
- cart: um usuário adicionou um produto ao carrinho de compras
- remove_from_cart: um usuário removeu um produto do carrinho de compras
- purchase: um usuário comprou um produto.

Várias compras por sessão
Uma sessão pode ter vários eventos de compra. Tudo bem, porque é um pedido único.

# **Entendimento dos dados - Visualização**

In [ ]:
# imports gerais 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime
import squarify
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from sklearn.utils import resample
from sklearn import metrics
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# O atributo shape retorna uma tupla, mostrando quantas linhas e colunas temos
df.shape

Esse dataset contém 9 colunas e 42.448.764 linhas.

In [ ]:
# retorna o número total de valores ausentes.
df.isnull().sum().sum()

Esse dataset contém 19.632.691 valores ausentes.

In [ ]:
df.isnull().sum()

Apenas 3 colunas tem dados faltantes.

In [ ]:
## tirar notação cientifica
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
#porcentagem de valores que estão faltando
df.isna().mean()

In [ ]:
# identificar o tipo de dados
df.dtypes

In [ ]:
# nomes das colunas
df.columns

In [ ]:
# Verificando detalhes estatísticos do dataset
df.describe().transpose()

Observa-se que nesse dataset, apenas a coluna preço é numerica. Mas por enquanto está considerando 'product_id',  'category_id' e 'user_id' como numerica. Vamos modificar isso.

In [ ]:
# convert column type
df = df.astype({"event_type": 'category', "product_id": 'category',
                "category_id": 'category', "category_code": 'category', "brand": 'category',
               "user_id": 'category', "user_session": 'category' })

In [ ]:
# identificar o tipo de dados
df.dtypes

In [ ]:
# Verificando detalhes estatísticos do dataset
df.describe().transpose()

Agora vemos que apenas a coluna 'price' é numérica.

In [ ]:
#coluna 'event_time' - data e hora, tirando UTC
df['event_time']=pd.to_datetime(df['event_time']).dt.tz_convert(None)

In [ ]:
df.head(3)

## **Gráficos**: 

1. Criando uma tabela de resumo para visão geral

In [ ]:
daily_summary_table = df.groupby(by=[df['event_time'].dt.normalize()]).agg(Number_of_daily_visits=('user_session',lambda x: x.nunique()),
                                                                                  Number_of_daily_visitors=('user_id',lambda x: x.nunique())
                                                                                  )
sales_filter = df['event_type']=='purchase'
sales = df.loc[sales_filter].groupby(by=[df['event_time'].dt.normalize()]).agg(number_of_daily_sales=('event_type','count'),
                                                                                      Total_daily_sales=('price','sum')
                                                                                      ).reset_index()
daily_summary_table = pd.merge(left=daily_summary_table,
                          right=sales,
                          left_on=['event_time'],
                          right_on=['event_time'],
                          how='left')
daily_summary_table['conversion_rate']=daily_summary_table['number_of_daily_sales']/daily_summary_table['Number_of_daily_visits']

In [ ]:
#Visitas Diárias
print('Estatísticas de visitas diárias')
print('-'*50)
print(daily_summary_table['Number_of_daily_visits'].describe())
print('-'*50)
print('Estatísticas de visitas por datas')
print('-'*50)
print(daily_summary_table.groupby(by=daily_summary_table['event_time'].dt.day_name())['Number_of_daily_visits'].describe())

In [ ]:
#Plotando o número de visitas diárias
fig=plt.figure(figsize=(18,9))
ax1=fig.add_subplot(2,1,1)
sns.lineplot(x='event_time',
              y='Number_of_daily_visits',
              data=daily_summary_table,
             ax=ax1)
plt.title('Visitas Diárias')
plt.ylabel('Número de visitas diárias')
plt.xlabel('Datas')

ax2=fig.add_subplot(2,1,2)

sns.boxplot(x=daily_summary_table['event_time'].dt.dayofweek,
            y='Number_of_daily_visits',
            data=daily_summary_table,
           ax=ax2)
plt.title('Número de visitas por dia da semana')
plt.ylabel('Número de visitas')
plt.xlabel('Dias')
plt.xticks([0, 1, 2,3,4,5,6], ['Segunda','Terça','Quarta','Quinta','Sexta','Sábado','Domingo'])
fig.tight_layout(pad=3.0);

2. Quantos clientes visitam o site?

In [ ]:
visitantes = df['user_id'].nunique()
print ("visitantes: {}".format(visitantes))

3. O tráfego de Visitantes flutua por data?

In [ ]:
d = df.loc[:,['event_time','user_id']]

In [ ]:
d['event_time'] = d['event_time'].apply(lambda s: str(s)[0:10])

In [ ]:
visitor_by_date = d.drop_duplicates().groupby(['event_time'])['user_id'].agg(['count']).sort_values(by=['event_time'], ascending=True)

In [ ]:
x = pd.Series(visitor_by_date.index.values).apply(lambda s: datetime.strptime(s, '%Y-%m-%d').date())
y = visitor_by_date['count']
plt.rcParams['figure.figsize'] = (20,8)

plt.title('Visitantes flutuando por data')
plt.plot(x,y)
plt.show()

4. Qual categoria os clientes interagem mais?

In [ ]:
top_category_n = 30
top_category = df.loc[:,'category_code'].value_counts()[:top_category_n].sort_values(ascending=False)
squarify.plot(sizes=top_category, label=top_category.index.array, color=["red","cyan","green","orange","blue","grey"], alpha=.7  )

plt.title('Qual categoria os clientes interagem mais?')
plt.axis('off')
plt.show()

5. Qual marca é mais vista?

In [ ]:
df['brand'].value_counts().head(50).plot.bar(figsize = (18, 7))
plt.title('Top Brand', fontsize = 20)
plt.xlabel('Names of Brand')
plt.ylabel('Count')
plt.show()

In [ ]:
#deletando para não ocupar memoria
del d

In [ ]:
#deletando para não ocupar memoria
del daily_summary_table

In [ ]:
#deletando para não ocupar memoria
del sales_filter

In [ ]:
#deletando para não ocupar memoria
del visitor_by_date

6. Qual item os clientes mais compram?

In [ ]:
# event_type = "purchase"
purchase = df.loc[df['event_type'] == 'purchase']
purchase = purchase.dropna(axis='rows')
purchase.head()

7. Quais marcas os clientes compram?

In [ ]:
top_sellers = purchase.groupby('brand')['brand'].agg(['count']).sort_values('count', ascending=False)
top_sellers.head(20)

In [ ]:
#deletando para não ocupar memoria
del purchase

In [ ]:
print('Total de marcas únicas : {} '.format(len(df['brand'].unique())))

## A variável alvo

Coluna  **event_type**: Representa o tipo de evento.

In [ ]:
df.groupby("event_type")["event_type"].count().sort_values(ascending=False)

Observando nossa target, vemos que é um problema de Classificação por existir vários valores categóricos.

In [ ]:
colors = ['yellowgreen', 'lightskyblue','lightcoral']
explode = [0, 0.1,0.1]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
axes[0].bar(df["event_type"].value_counts().index,df["event_type"].value_counts().values)
axes[1].pie(df["event_type"].value_counts().values, colors = colors, explode = explode, shadow = True, autopct = '%.2f%%')
axes[1].legend(df["event_type"].value_counts().index)

Nesse mês de Outubro, o cliente ou visualizou o produto ou adicionou o produto ao carrinho ou comprou o produto.

Nenhum usuário removeu produtos do carrinho de compras.

## **Preparação dos dados**



**Como a base está muito desbalanceada, vamos optar por excluir os visualizados e trabalhar apenas com produtos colocados no carrinho e comprados.**

In [ ]:
# Get indexes
index_View = df[ df['event_type'] == "view" ].index
# Delete these row indexes from dataFrame
df = df.drop(index_View)

In [ ]:
df.head(3)

In [ ]:
df.groupby("event_type")["event_type"].count().sort_values(ascending=False)

- Remover atributos não relevantes ao modelo

Vamos começar optando por excluir algumas colunas que não fazem parte do modelo. Primeiro a coluna 'user_id' pois trata-se apenas do ID do usuário e não faz sentido para o problema.



In [ ]:
df = df.drop(columns=['user_id'])

Vamos optar por remover também a coluna 'user_session' já que como o próprio nome diz é apenas um ID da sessão do usuário.

In [ ]:
df = df.drop(columns=['user_session'])

Agora vamos remover a coluna 'category_code', pois ela tem muitos dados faltantes. Além disso, ela é apenas uma Taxonomia da categoria do produto e já existe a coluna 'category_id' que representa o ID da categoria do produto. Ou seja, as 2 colunas entregam a mesma informação.

Mas antes de excluir definitivamente, vamos preencher os valores faltantes e extrair informações:

In [ ]:
# identificar o tipo de dados
df.dtypes

In [ ]:
df['category_code'] = df['category_code'].astype('string')

In [ ]:
df.update(df['category_code'].fillna('NaN.NaN'))

In [ ]:
df['category_code'] = df['category_code'].astype('category')

In [ ]:
#Conferindo se deu certo (NaN.NaN)
df.head(3)

Agora vamos extrair as informações dela gerando 2 colunas:

- category_code_level1: categoria

- category_code_level2: subcategoria

In [ ]:
#coluna 'category_code'
## código hierárquico - podemos dividir por dígitos em até 3 níveis
print(df["category_code"].sample(frac=0.01).nunique())
df["category_code"].value_counts()

In [ ]:
df["category_code_level1"] = df["category_code"].str.split(".",expand=True)[0].astype('category')

print(df["category_code_level1"].nunique())
df["category_code_level1"].value_counts()

In [ ]:
df["category_code_level2"] = df["category_code"].str.split(".",expand=True)[1].astype('category')

print(df["category_code_level2"].nunique())
df["category_code_level2"].value_counts()

In [ ]:
df.head(9)["category_code"].str.split(".",expand=True)[1]

In [ ]:
df = df.drop(columns=['category_code'])

In [ ]:
df.head(3)

In [ ]:
sum_result = df.isna().sum(axis=0).sort_values(ascending=False)
missing_values_columns = sum_result[sum_result > 0]
print('They are %s columns with missing values : \n%s' % (missing_values_columns.count(), [(index, value) for (index, value) in missing_values_columns.iteritems()]))


In [ ]:
# retorna o número total de valores ausentes
df.isnull().sum().sum()

Caiu para 77.111 valores ausentes.

- **Tratamento de dados faltantes**

Há apenas 1 coluna com dados faltantes: 'brand'.

In [ ]:
df.isnull().sum()

Como a princípio, a coluna 'brand' é muito importante para a decisão de compra de um produto, não podemos excluir. Dessa forma, vamos optar por substituir elas com um valor fixo, por exemplo ‘Marca não cadastrada’.

In [ ]:
df['brand'] = df['brand'].astype('string')

In [ ]:
df.update(df['brand'].fillna('Marca não cadastrada'))

In [ ]:
df['brand'] = df['brand'].astype('category')

In [ ]:
# Conferindo 'Marca não cadastrada'
df.head()

In [ ]:
#Zerando dados faltantes
df.isnull().sum().sum()

- Gerar atributos derivados

Vou adicionar alguns novos recursos ao conjunto de dados:


event_weekday: dia da semana do evento

dia

hora

minuto

segundo


is_purchased: se o item colocado no carrinho é comprado

In [ ]:
#dia da semana do evento
df['event_weekday'] = df['event_time'].apply(lambda s: str(datetime.strptime(str(s)[0:10], "%Y-%m-%d").weekday()))

In [ ]:
df.head(3)

In [ ]:
#ano não importa pois todos são do mesmo ano.
#mês também não importa pois todos são de Outubro. 
df.event_time = pd.to_datetime(df.event_time, utc=True)

df['day']=df['event_time'].dt.day
df['hour']=df['event_time'].dt.hour
df['minute']=df['event_time'].dt.minute
df['seconds']=df['event_time'].dt.second

In [ ]:
df.head(3)

In [ ]:
#excluindo coluna 'event_time' pois já extraimos o máximo de informações dela.
df = df.drop(columns=['event_time'])

In [ ]:
df.head(3)

In [ ]:
# se o item colocado no carrinho é comprado
# where filtra
# groupby agrupa
# transform faz com que o DataFrame produzido tenha o mesmo comprimento
df["is_purchased"] = np.where(df["event_type"]=="purchase",1,0)
#df["is_purchased"] = df.groupby(["user_session","product_id"])["is_purchased"].transform("max")

In [ ]:
#Vamos optar por não remover eventos iguais por primeiramente considerar todos importantes.
#drop_duplicates remove linhas duplicadas.
#df = df.loc[df["event_type"]=="cart"].drop_duplicates(["user_session","product_id","is_purchased"])

In [ ]:
df.head(3)

In [ ]:
# identificar o tipo de dados
df.dtypes

In [ ]:
#correlação
corr_mat = df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corr_mat, vmax=0.8, square=True)

## **Implementação e Modelagem**

Construção de modelos: Pycaret

In [ ]:
! pip install pycaret

In [ ]:
from pycaret import classification

In [ ]:
# O atributo shape retorna uma tupla, mostrando quantas linhas e colunas temos
df.shape

De 42.448.762 linhas, caiu para 1.669.365 linhas.

In [ ]:
#Esta função inicializa o ambiente em pycaret e cria o pipeline de transformação para preparar os dados para modelagem e implantação. 
# setup () deve ser chamado antes de executar qualquer outra função no pycaret.
# representa apenas 0,5% 
clf = classification.setup(df.head(8345), 
                           target = 'is_purchased',
                           categorical_features = ["product_id", "category_id", "category_code_level1",
                           "category_code_level2", "brand"],
                           numeric_features = ["price"],
                           silent = True,
                           fix_imbalance = False,
                           use_gpu = True, 
                           session_id = 786)

In [ ]:
# mostrar todos os modelos na biblioteca 
all_models = classification.models ()
print(all_models)

In [ ]:
best_model_1 = classification.compare_models(n_select=1,fold=3,
                                        cross_validation=False, sort = 'F1')

In [ ]:
print(best_model_1)

Baseado na métrica f1_score, o melhor modelo encontrado foi o LogisticRegression. 

In [ ]:
#Esta função cria um modelo e pontua-o usando Validação Cruzada Estratificada. 
#A saída imprime uma grade de pontuação que mostra Precisão, AUC, Recall, Precisão, F1, Kappa e MCC 
found_model_1 = classification.create_model('lr')

In [ ]:
#pip install statsmodels --upgrade

In [ ]:
#pip install scipy==1.2.0

In [ ]:
import statsmodels.formula.api as smf
from scipy import stats
import scipy
from scipy.special import factorial

In [ ]:
#pip install scipy==1.2 --upgrade

In [ ]:
#pip install --upgrade pip

Importancia das features

In [ ]:
#Esta função pega um objeto de modelo treinado e retorna um gráfico com base no conjunto de teste / manutenção
classification.plot_model(found_model_1, 'feature')

A feature mais importante segundo o modelo foi: 'event_type_cart', que significa colocar no carrinho.

## **Avaliação do modelo**

In [ ]:
classification.plot_model(found_model_1, 'auc')

In [ ]:
classification.plot_model(found_model_1, 'pr')

In [ ]:
classification.plot_model(found_model_1, 'confusion_matrix')

In [ ]:
classification.plot_model(found_model_1, 'error')

In [ ]:
classification.plot_model(found_model_1, 'class_report')

In [ ]:
classification.plot_model(found_model_1, 'parameter')

**Agora vamos testar sem a variável 'event_type_cart'.**

In [ ]:
#Esta função inicializa o ambiente em pycaret e cria o pipeline de transformação para preparar os dados para modelagem e implantação. 
# setup () deve ser chamado antes de executar qualquer outra função no pycaret.
# representa apenas 0,5%
clf = classification.setup(df.head(8345), 
                           target = 'is_purchased',
                           ignore_features = ['event_type'],
                           categorical_features = ["product_id", "category_id", "category_code_level1",
                           "category_code_level2", "brand"],
                           numeric_features = ["price"],
                           silent = True,
                           fix_imbalance = False,
                           use_gpu = True, 
                           session_id = 786)

In [ ]:
best_model = classification.compare_models(n_select=1,fold=3,
                                        cross_validation=False, sort = 'F1')

In [ ]:
print(best_model)

Baseado na métrica f1_score e ignorando a feature 'event_type', o melhor modelo encontrado foi RandomForestClassifier.

In [ ]:
found_model = classification.create_model('rf')

In [ ]:
# importancia das features
classification.plot_model(found_model, 'feature')

In [ ]:
# Avaliação
classification.plot_model(found_model, 'auc')

In [ ]:
classification.plot_model(found_model, 'pr')

In [ ]:
classification.plot_model(found_model, 'confusion_matrix')

In [ ]:
classification.plot_model(found_model, 'error')

## **DESCOBERTAS**:

Agora que já sabemos o melhor algoritmo, vamos aplicar na base inteira.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

In [ ]:
#ignorando a coluna
#df = df.drop('event_type', inplace=True, axis=1)
df = df.drop(columns=['event_type'])

In [ ]:
column_names = df.columns.values

In [ ]:
print(column_names)

In [ ]:
# column_names
print(column_names[-1])

In [ ]:
X = df.loc[:, column_names[:-1]]
y = df.loc[:,column_names[-1]]

In [ ]:
X.head(3)

In [ ]:
# Codificando a variável categórica presente em X
# columns [5,6,7] - NOT Binary vector
# Iriamos usar OneHotEncoder, no entanto não é recomendado para variáveis com mais de 15 valores diferentes.

from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
X[['product_id', 'category_id', 'brand', 'category_code_level1', 'category_code_level2', 'event_weekday']] = enc.fit_transform(X[['product_id', 'category_id', 'brand', 'category_code_level1', 'category_code_level2', 'event_weekday']])

In [ ]:
#divisao em treino e teste 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.2,
                                                   random_state = 0,
                                                   stratify=y)

In [ ]:
# visualizando o resultado
print('X_train:\n')
print(X_train)
print('\nX_test:\n')
print(X_test)

In [ ]:
X_train.head(3)

In [ ]:
#test_size = porcentagem para teste
#random_state = embaralha os dados(linhas), numero aleatorio
#stratify = manter a mesma proporcao de teste e treino

In [ ]:
# identificar o tipo de dados
X_train.dtypes

In [ ]:
# convert column type
X_train = X_train.astype({"product_id": object,
                "category_id": object, "brand": object, "category_code_level1": object, "category_code_level2": object})

In [ ]:
# identificar o tipo de dados
X_train.dtypes

In [ ]:
X_train.columns

Variáveis categóricas carregam informações qualitativas, ou seja, não possuem uma relação de magnitude, de $>, <, \leq, \geq$, como ocorre com variáveis numéricas. Devem ser, portanto, codificadas de forma que isso seja transferido para o modelo matemático.

Codificando a variavel categorica presente em X


In [ ]:
# Codificando a variável categórica presente em X
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.compose import ColumnTransformer

# columns [0] - Binary vector

#ct_X = ColumnTransformer(transformers=[('onehot',
#                                       OneHotEncoder(),
#                                       [0])],
#                        remainder='passthrough')

#X_train = ct_X.fit_transform(X_train)
#X_test = ct_X.transform(X_test)

Codificando a variável dependente y

In [ ]:
# Codificando a variável dependente y
from sklearn.preprocessing import LabelEncoder

In [ ]:
# aplicando LabelEncoder
labelencoder_y = LabelEncoder()
y_train = labelencoder_y.fit_transform(y_train)
y_test = labelencoder_y.transform(y_test)

In [ ]:
# Train a classifier model
classifier = RandomForestClassifier(bootstrap=True,
                                    ccp_alpha=0.0, class_weight=None,
                                    criterion='gini', max_depth=None, max_features='auto',
                                    max_leaf_nodes=None, max_samples=None,
                                    min_impurity_decrease=0.0, min_impurity_split=None,
                                    min_samples_leaf=1, min_samples_split=2,
                                    min_weight_fraction_leaf=0.0, n_estimators=100,
                                    n_jobs=-1, oob_score=False, random_state=786, verbose=0, warm_start=False)

classifier.fit(X_train, y_train)

In [ ]:
# predição no dataset de test
y_pred = classifier.predict(X_test)

In [ ]:
print('y_pred:\n')
print(y_pred)
print('\ny_test:\n')
print(y_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import recall_score

recall_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import precision_score

precision_score(y_test, y_pred)

In [ ]:
# matriz de confusão
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix=\n{}\n'.format(cm))

In [ ]:
# FPR
FPR = cm[0,1]/(sum(cm[0,:]))
print('FPR = %.3f \n' % FPR)

# TPR
TPR = cm[1,1]/(sum(cm[1,:]))
print('TPR = %.3f' % TPR)

In [ ]:
# roc curve parameters
from sklearn import metrics
prob = classifier.predict_proba(X_test)[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, prob)
roc_auc = metrics.auc(fpr, tpr)

In [ ]:
# ROC curve plot
fig, ax1 = plt.subplots()
ax1.plot(fpr, tpr, color='darkorange', lw=2, label='AUC (teste) = %0.2f' % roc_auc)
ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--',
         label='AUC (baseline) = 0.50')
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.01])
ax1.grid()
ax1.set_yticks(np.arange(0, 1, .1))
ax1.set_xticks(np.arange(0, 1, .1))
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('Curva ROC')
ax1.legend(loc="right")

# create the axis of thresholds (scores)
ax2 = ax1.twinx()
ax2.plot(fpr, threshold, markeredgecolor='r',linestyle='dashed', color='r')
ax2.set_yticks(np.arange(0, 2, .1))
ax2.set_ylabel('Threshold',color='r')
ax2.set_ylim([threshold[-1],threshold[0]])
ax2.set_xlim([fpr[0],fpr[-1]])

plt.show()

In [ ]:
classifier.feature_importances_

In [ ]:
feature_names = X_train.columns
print(feature_names)

In [ ]:
plt.barh(feature_names, classifier.feature_importances_)

In [ ]:
sorted_idx = classifier.feature_importances_.argsort()
plt.barh(feature_names[sorted_idx], classifier.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")